In [1]:
# Constants
n_aisle = 5 # number of aisles
l_aisle = 20 # length 
n_robots = 3

# Speeds m/s
fwd_speed = 1.95
bck_speed = 0.75
rot_speed = 0.45

cell_size = 2.9 # horizontal distance b/w 2 shelves
fwd_time = cell_size / fwd_speed
bck_time = cell_size / bck_speed

shelf_height = 6 # max levels (1 = ground)
shelf_time = 3.0 # time to go up / down one level
pick_time = 3.0 # time to pick a pallette in/out

# Level start from 1 !!
function shelf_pickup(level::UInt16)::Float32
    return 2 * shelf_time * (level - 1) + pick_time 
end

shelf_pickup (generic function with 1 method)

In [1]:
@enum Direction Up Down Left Right

# Any "geolocated" object
abstract type AbstractLocatedObject end

# 32,768 should be enough for position
struct AGV <: AbstractLocatedObject
    x::Int16
    y::Int16
    dir::Direction
    hasPalette::Bool
    
    AGV(x, y, d, p) = x < 1 || x > 2 * n_aisle || 
                      y < -1 || y > l_aisle + 1 ? error("Coordinates out of range") : new(x, y, d, p)  
end

struct Palette <: AbstractLocatedObject
    x::Int16
    y::Int16
    level::UInt16
    
    Palette(x, y, level) = x < 1 || x > 2 * n_aisle - 1 || 
                           y < 1 || y > l_aisle ||     
                          level < 1 || level > shelf_height ? error("Values out of range") : new(x, y, level)  
end

struct FloorPosition <: AbstractLocatedObject
    x::Int16
    y::Int16
    FloorPosition(x, y) = x < 1  || x > 2 * n_aisle || 
                          y < -1 || y > l_aisle + 1 ? error("Coordinates out of range") : new(x, y)  
end

In [1]:
Conv_1 = FloorPosition(1, -1) # Conveyor 1
Exit = FloorPosition(2 * n_aisle - 1, -1) # Delivery point - assume shelf width = 1

FloorPosition(9, -1)

Time is the measure/cost by which everything is measured.


In [1]:
# Generic function depending on the location considered
function time_to_point(robot::AGV, palette::Palette) 
    return time_to_palette(robot::AGV, palette::Palette)
end

# Time to a palette. Time is identical to load/unload

# Returns the distance (in time from robot to palette) + direction in which to starts the movement
function time_to_palette(robot::AGV, palette::Palette)
    
    # If robot already carries a palette, extremely high distance to never target one.
    if robot.hasPalette == true
        return (1.0f9, robot.dir)
    end
    
    # The robot can travel all the way forward or backward
    # and going up or down
    
    la = Float32(l_aisle)
    rx = Float32(robot.x)
    ry = Float32(robot.y)
    px = Float32(palette.x)
    py = Float32(palette.y)

    # x time is just number of cells x time per cell
    fwd_t_x = 2 * abs(rx - px) * fwd_time
    bck_t_x = 2 * abs(rx - px) * bck_time

    # print(robot.x, " ", palette.x, " ", fwd_t_x, " ", bck_t_x, "\n")
    
    #  y time depends on the direction faced 
    # Final time is sum of all + time to get palette
    if robot.dir == Up 
        
        # If in the same aisle in front of robot, fwd short, bck full circle 
        # crossing next aisle = 2, entering/exiting aisle = 1
        if (robot.x == palette.x) && (palette.y >= robot.y)
            fwd_t_y = (py - ry) * fwd_time
            bck_t_y = ((ry - 1) + 1 + 2 + 1 + (la - 1) + 1 + 2 + 1 + la - py) * bck_time
            
        # If in the same aisle behind robot => full circle 
        elseif (robot.x == palette.x) && (palette.y < robot.y)
            fwd_t_y = ((la - ry) + 1 + 2 + 1 + (la - 1) + 1 + 2 + 1 + (py - 1)) * fwd_time
            bck_t_y = (ry - py) * bck_time
            
        else
            # Going fwd pointing up: to go to last cell in aisle + 1 to horizontal lane + down to palette
            # Going bck pointing up: opposite trajectory
            fwd_t_y = (la - ry + 1 + 2 + 1 + la - py) * fwd_time
            bck_t_y = ((ry - 1) + 1 + 2 + 1 + (py - 1)) * bck_time
        end
        
        fwd_total = fwd_t_x + fwd_t_y + shelf_pickup(palette.level)
        bck_total = bck_t_x + bck_t_y + shelf_pickup(palette.level)
        
        # print(robot.dir, " ", fwd_t_y, " ", bck_t_y, " ", fwd_up_total, " ", bck_down_total)
        
        if fwd_total <= bck_total
            return (fwd_total, Up)
        else
            return (bck_total, Down)
        end
        
    else # If robot facing down
        if (robot.x == palette.x) & (palette.y <= robot.y)
            fwd_t_y = (ry - py) * fwd_time
            bck_t_y = ((la - ry) + 1 + 2 + 1 + (la - 1) + 1 + 2 + 1 + (py - 1)) * bck_time
            
        # If in the same aisle behind robot => full circle 
        elseif (robot.x == palette.x) & (palette.y > robot.y)
            fwd_t_y = ((ry - 1) + 1 + 2 + 1 + (la - 1) + 1 + 2 + 1 + (la - py)) * fwd_time
            bck_t_y = (py - ry) * bck_time
            
        else
            # Going fwd pointing up: to go to last cell in aisle + 1 to horizontal lane + down to palette
            # Going bck pointing up: opposite trajectory
            fwd_t_y = (la - ry + 1 + 2 + 1 + la - py) * fwd_time
            bck_t_y = ((ry - 1) + 1 + 2 + 1 + (py - 1)) * bck_time
        end
        
        fwd_total = fwd_t_x + fwd_t_y + shelf_pickup(palette.level)
        bck_total = bck_t_x + bck_t_y + shelf_pickup(palette.level)
        
        # print(robot.dir, " ", fwd_t_y, " ", bck_t_y, " ", fwd_up_total, " ", bck_down_total)
        
        if fwd_total <= bck_total
            return (fwd_total, Down)
        else
            return (bck_total, Up)
        end
        
    end
        
end

time_to_palette (generic function with 1 method)

In [1]:
time_to_point(AGV(1, 4, Up, true), Palette(1, 5, 4))

(1.0f9, Up)

In [1]:
time_to_point(AGV(1, 4, Up, false), Palette(1, 5, 4))

(22.48717948717949, Up)

In [1]:
time_to_point(AGV(1, 4, Up, false), Palette(1, 3, 4))

(24.866666666666667, Down)

In [1]:
time_to_point(AGV(4, 4, Up, false), Palette(2, 3, 4))

(71.26666666666667, Down)

In [1]:
function time_to_point(robot::AGV, location::FloorPosition)
    return time_to_floor_location(robot::AGV, location::FloorPosition)
end

# TODO: Ne marche pas si robot vient juste de recuperer une palette. 
# Le choix de direction n'est pas encore choisie.

# Returns the distance (in time from robot to palette) + direction in which to starts the movement
function time_to_floor_location(robot::AGV, location::FloorPosition)
    
    la = Float32(l_aisle)
    rx = Float32(robot.x)
    ry = Float32(robot.y)
    lx = Float32(location.x)
    ly = Float32(location.y)

    # The robot can travel all the way forward or backward
    # and going up or down
    
    # x time is just number of cells x time per cell
    fwd_t_x = 2 * abs(rx - lx) * fwd_time
    bck_t_x = 2 * abs(rx - lx) * bck_time

    # print(robot.x, " ", palette.x, " ", fwd_t_x, " ", bck_t_x, "\n")
    
    #  y time depends on the direction faced 
    # Final time is sum of all + time to get palette
    if robot.dir == Up 
        
        # If in the same aisle in front of robot, fwd short, bck full circle 
        # crossing next aisle = 2, entering/exiting aisle = 1
        if (robot.x == palette.x) && (palette.y >= robot.y)
            fwd_t_y = (py - ry) * fwd_time
            bck_t_y = ((ry - 1) + 1 + 2 + 1 + (la - 1) + 1 + 2 + 1 + la - py) * bck_time
            
        # If in the same aisle behind robot => full circle 
        elseif (robot.x == palette.x) && (palette.y < robot.y)
            fwd_t_y = ((la - ry) + 1 + 2 + 1 + (la - 1) + 1 + 2 + 1 + (py - 1)) * fwd_time
            bck_t_y = (ry - py) * bck_time
            
        else
            # Going fwd pointing up: to go to last cell in aisle + 1 to horizontal lane + down to palette
            # Going bck pointing up: opposite trajectory
            fwd_t_y = (la - ry + 1 + 2 + 1 + la - py) * fwd_time
            bck_t_y = ((ry - 1) + 1 + 2 + 1 + (py - 1)) * bck_time
        end
        
        fwd_total = fwd_t_x + fwd_t_y 
        bck_total = bck_t_x + bck_t_y 
        
        # print(robot.dir, " ", fwd_t_y, " ", bck_t_y, " ", fwd_up_total, " ", bck_down_total)
        
        if fwd_total <= bck_total
            return (fwd_total, Up)
        else
            return (bck_total, Down)
        end
        
    else # If robot facing down
        if (robot.x == palette.x) & (palette.y <= robot.y)
            fwd_t_y = (ry - py) * fwd_time
            bck_t_y = ((la - ry) + 1 + 2 + 1 + (la - 1) + 1 + 2 + 1 + (py - 1)) * bck_time
            
        # If in the same aisle behind robot => full circle 
        elseif (robot.x == palette.x) & (palette.y > robot.y)
            fwd_t_y = ((ry - 1) + 1 + 2 + 1 + (la - 1) + 1 + 2 + 1 + (la - py)) * fwd_time
            bck_t_y = (py - ry) * bck_time
            
        else
            # Going fwd pointing up: to go to last cell in aisle + 1 to horizontal lane + down to palette
            # Going bck pointing up: opposite trajectory
            fwd_t_y = (la - ry + 1 + 2 + 1 + la - py) * fwd_time
            bck_t_y = ((ry - 1) + 1 + 2 + 1 + (py - 1)) * bck_time
        end
        
        fwd_total = fwd_t_x + fwd_t_y 
        bck_total = bck_t_x + bck_t_y 
        
        # print(robot.dir, " ", fwd_t_y, " ", bck_t_y, " ", fwd_up_total, " ", bck_down_total)
        
        if fwd_total <= bck_total
            return (fwd_total, Down)
        else
            return (bck_total, Up)
        end
        
    end       
end

time_to_floor_location (generic function with 1 method)

In [1]:
time_to_point(AGV(4, 2, Up, true), Exit)

Error: [object Object]

In [1]:
distance_to_exit(AGV(4, 2, Down, true))

Down 4.461538461538462 166.26666666666668 19.333333333333336 204.93333333333334

(19.333333333333336, Down)